In [1]:
from reactionmodel.model import Species, Reaction, ReactionRateFamily, Model
import reactionmodel.smallparser as smallparser
import yaml

In [2]:
from yaml import SafeLoader as Loader

yaml_string = """
families:
  As: [x, y, z]
  Bs: [p, q, r]

species:
- name: 'A_$i'
  used_families: {'i': 'As'}
- name: 'B_$j'
  used_families: {'j': 'Bs'}

reactions:
- description: 2A_$i => B_$j
  reactants: [['A_$i',2]]
  products: ['B_$j']
  k: 'Q[#i][#j]'
  used_families: {'i': 'As', 'j': 'Bs'}

parameters:
 foo: 2.0
 Q: {'path': './examples/minimal/q.csv'}

simulator_config:
  simulator: gillespie

initial_condition:
  A_$i: 1.0 * foo
  used_families: {'i': 'As'}
"""

md = yaml.load(yaml_string, Loader=Loader)
md

{'families': {'As': ['x', 'y', 'z'], 'Bs': ['p', 'q', 'r']},
 'species': [{'name': 'A_$i', 'used_families': {'i': 'As'}},
  {'name': 'B_$j', 'used_families': {'j': 'Bs'}}],
 'reactions': [{'description': '2A_$i => B_$j',
   'reactants': [['A_$i', 2]],
   'products': ['B_$j'],
   'k': 'Q[#i][#j]',
   'used_families': {'i': 'As', 'j': 'Bs'}}],
 'parameters': {'foo': 2.0, 'Q': {'path': './examples/minimal/q.csv'}},
 'simulator_config': {'simulator': 'gillespie'},
 'initial_condition': {'A_$i': '1.0 * foo', 'used_families': {'i': 'As'}}}

In [3]:
p = smallparser.loads(md)

{'species': [{'name': 'A_x'}, {'name': 'A_y'}, {'name': 'A_z'}, {'name': 'B_p'}, {'name': 'B_q'}, {'name': 'B_r'}], 'reactions': [{'description': '2A_x => B_p', 'reactants': [['A_x', 2]], 'products': ['B_p'], 'k': 'Q[0][0]'}, {'description': '2A_x => B_q', 'reactants': [['A_x', 2]], 'products': ['B_q'], 'k': 'Q[0][1]'}, {'description': '2A_x => B_r', 'reactants': [['A_x', 2]], 'products': ['B_r'], 'k': 'Q[0][2]'}, {'description': '2A_y => B_p', 'reactants': [['A_y', 2]], 'products': ['B_p'], 'k': 'Q[1][0]'}, {'description': '2A_y => B_q', 'reactants': [['A_y', 2]], 'products': ['B_q'], 'k': 'Q[1][1]'}, {'description': '2A_y => B_r', 'reactants': [['A_y', 2]], 'products': ['B_r'], 'k': 'Q[1][2]'}, {'description': '2A_z => B_p', 'reactants': [['A_z', 2]], 'products': ['B_p'], 'k': 'Q[2][0]'}, {'description': '2A_z => B_q', 'reactants': [['A_z', 2]], 'products': ['B_q'], 'k': 'Q[2][1]'}, {'description': '2A_z => B_r', 'reactants': [['A_z', 2]], 'products': ['B_r'], 'k': 'Q[2][2]'}]}
[Reac

In [4]:
p.initial_condition

{'A_x': '1.0 * foo', 'A_y': '1.0 * foo', 'A_z': '1.0 * foo'}

In [5]:
p.model.make_initial_condition(p.initial_condition, p.parameters)

Evaluating expression: 1.0 * foo => 2.0
Evaluating expression: 1.0 * foo => 2.0
Evaluating expression: 1.0 * foo => 2.0


array([2., 2., 2., 0., 0., 0.])

In [6]:
p.parameters

{'foo': 2.0,
 'Q': array([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])}

In [7]:
p.model.get_k(parameters=p.parameters)

Evaluating expression: Q[0][0] => 1.0
Evaluating expression: Q[0][1] => 2.0
Evaluating expression: Q[0][2] => 3.0
Evaluating expression: Q[1][0] => 4.0
Evaluating expression: Q[1][1] => 5.0
Evaluating expression: Q[1][2] => 6.0
Evaluating expression: Q[2][0] => 7.0
Evaluating expression: Q[2][1] => 8.0
Evaluating expression: Q[2][2] => 9.0


array([1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [8]:
p.simulator_config

{'simulator': 'gillespie'}

In [9]:
p.model.all_reactions

[Reaction(description=2A_x => B_p, reactants=((Species(name='A_x', description=''), 2),), products=(Species(name='B_p', description=''),), kinetic_order=(), k=Q[0][0]),
 Reaction(description=2A_x => B_q, reactants=((Species(name='A_x', description=''), 2),), products=(Species(name='B_q', description=''),), kinetic_order=(), k=Q[0][1]),
 Reaction(description=2A_x => B_r, reactants=((Species(name='A_x', description=''), 2),), products=(Species(name='B_r', description=''),), kinetic_order=(), k=Q[0][2]),
 Reaction(description=2A_y => B_p, reactants=((Species(name='A_y', description=''), 2),), products=(Species(name='B_p', description=''),), kinetic_order=(), k=Q[1][0]),
 Reaction(description=2A_y => B_q, reactants=((Species(name='A_y', description=''), 2),), products=(Species(name='B_q', description=''),), kinetic_order=(), k=Q[1][1]),
 Reaction(description=2A_y => B_r, reactants=((Species(name='A_y', description=''), 2),), products=(Species(name='B_r', description=''),), kinetic_order=()

In [10]:
p

ParseResults(model=<reactionmodel.model.Model object at 0x12cf21340>, parameters={'foo': 2.0, 'Q': array([[1., 2., 3.],
       [4., 5., 6.],
       [7., 8., 9.]])}, initial_condition={'A_x': '1.0 * foo', 'A_y': '1.0 * foo', 'A_z': '1.0 * foo'}, simulator_config={'simulator': 'gillespie'})

In [11]:
a = Species('A')
r1 = Reaction([a], [])
r2 = Reaction([(a, 2)], [])
r_g = ReactionRateFamily([r1, r2], k='hi')

m = Model([a], [r1,r2])
m.to_dict()
from yaml import SafeDumper as Dumper
dump = yaml.dump(m.to_dict(), Dumper=Dumper)

In [12]:
p = smallparser.loads(yaml.load(dump, Loader=Loader))

{'species': [{'description': '', 'name': 'A'}], 'reactions': [{'description': '', 'k': None, 'products': [], 'rate_involved': [{'description': '', 'name': 'A'}], 'reactants': [{'description': '', 'name': 'A'}], 'reversible': False}, {'description': '', 'k': None, 'products': [], 'rate_involved': [[{'description': '', 'name': 'A'}, 2]], 'reactants': [[{'description': '', 'name': 'A'}, 2]], 'reversible': False}]}
[Reaction(description=, reactants=(Species(name='A', description=''),), products=(), kinetic_order=(Species(name='A', description=''),), k=None), Reaction(description=, reactants=((Species(name='A', description=''), 2),), products=(), kinetic_order=((Species(name='A', description=''), 2),), k=None)]


In [13]:
p.model == m

True